In [15]:
from pyspark.sql import Window

import os
from dotenv import load_dotenv
load_dotenv()
import requests
from spark_common import initiate_spark,terminate_spark, get_circle_data , get_process_sensor, get_data_from_api,reading_and_validate_data,train_test_split
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType, IntegerType
from pyspark.sql.functions import *

import pandas as pd
import sys
from queue import Queue
from threading import Thread

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
import joblib
from pyspark.sql.functions import pandas_udf, PandasUDFType
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV


In [9]:
terminate_spark(spark)

In [12]:
spark = initiate_spark()
spark

In [4]:

circle_list = get_circle_data()
# data = get_process_sensor(circle_list[0]['id'])
data= []
for id in range(len(circle_list)):
    data_complete  = get_process_sensor(circle_list[id]['id'])
    data.extend(data_complete )

23349
20061
7354
21856
9748
16430
18322
17211
18589
20236
14214


In [13]:
schema_file_path = "schema.yaml"
# Validate the data against the schema
validated_df = reading_and_validate_data(spark, data, schema_file_path)

# Check if validation was successful
if validated_df is not None:
    df = validated_df
    # df = validated_df.select("creation_time","sensor_id","consumed_unit")
    # print(df.count())

else:
    print("Data validation failed.")


In [14]:
df.count()

187370

In [16]:
indexer = StringIndexer(inputCol="sensor_id", outputCol="encoded_sensor_id")
encoded_df = indexer.fit(df).transform(df)
numbers_to_partitions = encoded_df.select("encoded_sensor_id").distinct().count()
print(encoded_df.select("sensor_id").distinct().count())
print(numbers_to_partitions)

83
83


In [17]:
df = encoded_df.drop("sensor_id")
num = df.rdd.getNumPartitions()
print("number of partion before:",num)
repartitoned_df = df.repartition(numbers_to_partitions,['sensor_id'])
num1 = repartitoned_df.rdd.getNumPartitions()
print("number of partion after:",num1)

number of partion before: 8
number of partion after: 83


In [18]:
def split_train_test(key,pdf):
    total_rows = len(pdf)
    train_rows = int(0.8 * total_rows)
    pdf = pdf.sort_values(by="creation_time")
    train_df = pdf.iloc[:train_rows]
    FEATURES = ['is_holiday','temperature_2m', 'relative_humidity_2m', 'apparent_temperature','precipitation', 'wind_speed_10m','wind_speed_100m', 'lag1', 'lag2','lag3', 'lag4', 'lag5', 'day', 'hour', 'month', 'dayofweek', 'quarter','dayofyear', "encoded_sensor_id",'weekofyear', 'year']
    TARGET = ['consumed_unit']
    
    X_train = train_df[FEATURES]
    y_train = train_df[TARGET]

    # from xgboost import XGBRegressor
    xgb_model = XGBRegressor()
    xgb_model.fit(X_train, y_train)
    param_grid = {
    'n_estimators': [50, 100, 150, 200, 500],
    'max_depth': [5, 10, 15, 20, 30],
    'learning_rate': [0.01, 0.1, 0.3, 0.5, 0.7],
    'reg_alpha':  [0.001, 0.01, 0.1, 0.5, 1.0]
    }

    random_search = RandomizedSearchCV(xgb_model,
                                        param_distributions=param_grid,
                                        n_iter=10,
                                        scoring='neg_mean_squared_error',
                                        cv=5,
                                        # verbose=1,
                                        n_jobs=-1,
                                        random_state=45)

    # Fit the RandomizedSearchCV to the data
    random_search.fit(X_train, y_train)

    # Get the best parameters
    best_params = random_search.best_params_
    # print(f"Best Parameters for sensor {i}: {best_params}")

    # Train the model with the best parameters
    best_xgb_model = XGBRegressor(n_estimators=best_params['n_estimators'],
                                max_depth=best_params['max_depth'],
                                learning_rate=best_params['learning_rate'],
                                reg_alpha=best_params['reg_alpha'],
                                reg_lambda=0.01,
                                )
    best_xgb_model.fit(X_train, y_train)
    model_filename = f"xgb_model_sensor_{key}.joblib"
    joblib.dump(xgb_model, model_filename)
    return X_train
# result = (store_part.groupBy("sensor_id").apply(train_test_split))

In [19]:
result_schema = StructType([
            StructField("temperature_2m", FloatType(), True),
            StructField("relative_humidity_2m", IntegerType(), True),
            StructField("apparent_temperature", FloatType(), True),
            StructField("precipitation", FloatType(), True),
            StructField("wind_speed_10m", FloatType(), True),
            StructField("wind_speed_100m", FloatType(), True),
            StructField("is_holiday", IntegerType(), True),
            StructField("lag1", FloatType(), True),
            StructField("lag2", FloatType(), True),
            StructField("lag3", FloatType(), True),
            StructField("lag4", FloatType(), True),
            StructField("lag5", FloatType(), True),
            StructField("day", IntegerType(), True),
            StructField("hour", IntegerType(), True),
            StructField("month", IntegerType(), True),
            StructField("dayofweek", IntegerType(), True),
            StructField("quarter", IntegerType(), True),
            StructField("dayofyear", IntegerType(), True),
            StructField("encoded_sensor_id", FloatType(), True),
            StructField("weekofyear", IntegerType(), True),
            StructField("year", IntegerType(), True),
        ])

In [25]:
result = repartitoned_df.groupBy("encoded_sensor_id").applyInPandas(split_train_test,schema=result_schema)
result.show()

Py4JJavaError: An error occurred while calling o476.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 42.0 failed 1 times, most recent failure: Lost task 0.0 in stage 42.0 (TID 423) (host.docker.internal executor driver): java.lang.UnsupportedOperationException: sun.misc.Unsafe or java.nio.DirectByteBuffer.<init>(long, int) not available
	at org.apache.arrow.memory.util.MemoryUtil.directBuffer(MemoryUtil.java:174)
	at org.apache.arrow.memory.ArrowBuf.getDirectBuffer(ArrowBuf.java:229)
	at org.apache.arrow.memory.ArrowBuf.nioBuffer(ArrowBuf.java:224)
	at org.apache.arrow.vector.ipc.WriteChannel.write(WriteChannel.java:133)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.writeBatchBuffers(MessageSerializer.java:303)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.serialize(MessageSerializer.java:276)
	at org.apache.arrow.vector.ipc.ArrowWriter.writeRecordBatch(ArrowWriter.java:147)
	at org.apache.arrow.vector.ipc.

In [32]:
pdf = pd.DataFrame(data)
pdf

,temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,wind_speed_10m,wind_speed_100m,creation_time,sensor_id,count,is_holiday,...,lag6,lag7,day,hour,month,dayofweek,quarter,dayofyear,weekofyear,year
0,30.7,33,31.7,0.0,11.0,13.6,2024-03-22 08:00:00,022040a0-beb4-413c-bc3b-e6564fc2699c,9,0,...,NaN,NaN,22,8,3,4,1,82,12,2024
1,30.7,33,31.7,0.0,11.0,13.6,2024-03-22 08:15:00,022040a0-beb4-413c-bc3b-e6564fc2699c,10,0,...,NaN,NaN,22,8,3,4,1,82,12,2024
2,30.7,33,31.7,0.0,11.0,13.6,2024-03-22 08:30:00,022040a0-beb4-413c-bc3b-e6564fc2699c,9,0,...,NaN,NaN,22,8,3,4,1,82,12,2024
3,30.7,33,31.7,0.0,11.0,13.6,2024-03-22 08:45:00,022040a0-beb4-413c-bc3b-e6564fc2699c,9,0,...,NaN,NaN,22,8,3,4,1,82,12,2024
4,31.1,34,32.0,0.0,11.2,13.8,2024-03-22 09:00:00,022040a0-beb4-413c-bc3b-e6564fc2699c,9,0,...,NaN,NaN,22,9,3,4,1,82,12,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187365,26.1,34,24.3,0.0,10.9,24.8,2024-03-29 22:00:00,00a719bb-2814-4ecf-814d-7ac4f862e834,9,1,...,1.0,NaN,29,22,3,4,1,89,13,2024
187366,26.1,34,24.3,0.0,10.9,24.8,2024-03-29 22:15:00,00a719bb-2814-4ecf-814d-7ac4f862e834,9,1,...,0.5,NaN,29,22,3,4,1,89,13,2024
187367,26.1,34,24.3,0.0,10.9,24.8,2024-03-29 22:30:00,00a719bb-2814-4ecf-814d-7ac4f862e834,9,1,...,0.5,NaN,29,22,3,4,1,89,13,2024
187368,26.1,34,24.3,0.0,10.9,24.8,2024-03-29 22:45:00,00a719bb-2814-4ecf-814d-7ac4f862e834,9,1,...,0.5,NaN,29,22,3,4,1,89,13,2024


In [39]:
pdf

,temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,wind_speed_10m,wind_speed_100m,creation_time,sensor_id,count,is_holiday,...,lag6,lag7,day,hour,month,dayofweek,quarter,dayofyear,weekofyear,year
0,30.7,33,31.7,0.0,11.0,13.6,2024-03-22 08:00:00,022040a0-beb4-413c-bc3b-e6564fc2699c,9,0,...,NaN,NaN,22,8,3,4,1,82,12,2024
1,30.7,33,31.7,0.0,11.0,13.6,2024-03-22 08:15:00,022040a0-beb4-413c-bc3b-e6564fc2699c,10,0,...,NaN,NaN,22,8,3,4,1,82,12,2024
2,30.7,33,31.7,0.0,11.0,13.6,2024-03-22 08:30:00,022040a0-beb4-413c-bc3b-e6564fc2699c,9,0,...,NaN,NaN,22,8,3,4,1,82,12,2024
3,30.7,33,31.7,0.0,11.0,13.6,2024-03-22 08:45:00,022040a0-beb4-413c-bc3b-e6564fc2699c,9,0,...,NaN,NaN,22,8,3,4,1,82,12,2024
4,31.1,34,32.0,0.0,11.2,13.8,2024-03-22 09:00:00,022040a0-beb4-413c-bc3b-e6564fc2699c,9,0,...,NaN,NaN,22,9,3,4,1,82,12,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187365,26.1,34,24.3,0.0,10.9,24.8,2024-03-29 22:00:00,00a719bb-2814-4ecf-814d-7ac4f862e834,9,1,...,1.0,NaN,29,22,3,4,1,89,13,2024
187366,26.1,34,24.3,0.0,10.9,24.8,2024-03-29 22:15:00,00a719bb-2814-4ecf-814d-7ac4f862e834,9,1,...,0.5,NaN,29,22,3,4,1,89,13,2024
187367,26.1,34,24.3,0.0,10.9,24.8,2024-03-29 22:30:00,00a719bb-2814-4ecf-814d-7ac4f862e834,9,1,...,0.5,NaN,29,22,3,4,1,89,13,2024
187368,26.1,34,24.3,0.0,10.9,24.8,2024-03-29 22:45:00,00a719bb-2814-4ecf-814d-7ac4f862e834,9,1,...,0.5,NaN,29,22,3,4,1,89,13,2024


In [42]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Fit the LabelEncoder to the 'sensor_id' column and transform it
pdf['encoded_sensor_id'] = label_encoder.fit_transform(pdf['sensor_id'])

# Display the encoded DataFrame
encoded_df



array([59, 59, 59, ..., 19, 19, 19])

In [43]:
pdf

,temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,wind_speed_10m,wind_speed_100m,creation_time,sensor_id,count,is_holiday,...,lag7,day,hour,month,dayofweek,quarter,dayofyear,weekofyear,year,encoded_sensor_id
0,30.7,33,31.7,0.0,11.0,13.6,2024-03-22 08:00:00,022040a0-beb4-413c-bc3b-e6564fc2699c,9,0,...,NaN,22,8,3,4,1,82,12,2024,59
1,30.7,33,31.7,0.0,11.0,13.6,2024-03-22 08:15:00,022040a0-beb4-413c-bc3b-e6564fc2699c,10,0,...,NaN,22,8,3,4,1,82,12,2024,59
2,30.7,33,31.7,0.0,11.0,13.6,2024-03-22 08:30:00,022040a0-beb4-413c-bc3b-e6564fc2699c,9,0,...,NaN,22,8,3,4,1,82,12,2024,59
3,30.7,33,31.7,0.0,11.0,13.6,2024-03-22 08:45:00,022040a0-beb4-413c-bc3b-e6564fc2699c,9,0,...,NaN,22,8,3,4,1,82,12,2024,59
4,31.1,34,32.0,0.0,11.2,13.8,2024-03-22 09:00:00,022040a0-beb4-413c-bc3b-e6564fc2699c,9,0,...,NaN,22,9,3,4,1,82,12,2024,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187365,26.1,34,24.3,0.0,10.9,24.8,2024-03-29 22:00:00,00a719bb-2814-4ecf-814d-7ac4f862e834,9,1,...,NaN,29,22,3,4,1,89,13,2024,19
187366,26.1,34,24.3,0.0,10.9,24.8,2024-03-29 22:15:00,00a719bb-2814-4ecf-814d-7ac4f862e834,9,1,...,NaN,29,22,3,4,1,89,13,2024,19
187367,26.1,34,24.3,0.0,10.9,24.8,2024-03-29 22:30:00,00a719bb-2814-4ecf-814d-7ac4f862e834,9,1,...,NaN,29,22,3,4,1,89,13,2024,19
187368,26.1,34,24.3,0.0,10.9,24.8,2024-03-29 22:45:00,00a719bb-2814-4ecf-814d-7ac4f862e834,9,1,...,NaN,29,22,3,4,1,89,13,2024,19


In [44]:
grouped_df = pdf.groupby(['encoded_sensor_id'])

In [70]:
i = 0
model_dict = {}
for id , data in grouped_df:
    print(id[0])
    df = data.copy()
    model = split_train_test(pdf=df)
    print(len(df))
    model_dict[id[0]] = model
    i+=1
    if i>3:
        break
print(i)

0
1230
1
2044
2
3080
3
2372
4


In [71]:
model_dict

{0: XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...),
 1: XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False

In [48]:
def split_train_test(pdf):
    total_rows = len(pdf)
    train_rows = int(0.8 * total_rows)
    pdf = pdf.sort_values(by="creation_time")
    train_df = pdf.iloc[:train_rows]
    FEATURES = ['is_holiday','temperature_2m', 'relative_humidity_2m', 'apparent_temperature','precipitation', 'wind_speed_10m','wind_speed_100m', 'lag1', 'lag2','lag3', 'lag4', 'lag5', 'day', 'hour', 'month', 'dayofweek', 'quarter','dayofyear', "encoded_sensor_id",'weekofyear', 'year']
    TARGET = ['consumed_unit']
    
    X_train = train_df[FEATURES]
    y_train = train_df[TARGET]

    # from xgboost import XGBRegressor
    xgb_model = XGBRegressor()
    xgb_model.fit(X_train, y_train)
    param_grid = {
    'n_estimators': [50, 100, 150, 200, 500],
    'max_depth': [5, 10, 15, 20, 30],
    'learning_rate': [0.01, 0.1, 0.3, 0.5, 0.7],
    'reg_alpha':  [0.001, 0.01, 0.1, 0.5, 1.0]
    }

    random_search = RandomizedSearchCV(xgb_model,
                                        param_distributions=param_grid,
                                        n_iter=10,
                                        scoring='neg_mean_squared_error',
                                        cv=5,
                                        # verbose=1,
                                        n_jobs=-1,
                                        random_state=45)

    # Fit the RandomizedSearchCV to the data
    random_search.fit(X_train, y_train)

    # Get the best parameters
    best_params = random_search.best_params_
    # print(f"Best Parameters for sensor {i}: {best_params}")

    # Train the model with the best parameters
    best_xgb_model = XGBRegressor(n_estimators=best_params['n_estimators'],
                                max_depth=best_params['max_depth'],
                                learning_rate=best_params['learning_rate'],
                                reg_alpha=best_params['reg_alpha'],
                                reg_lambda=0.01,
                                )
    model = best_xgb_model.fit(X_train, y_train)
    return model
# result = (store_part.groupBy("sensor_id").apply(train_test_split))

In [58]:
model.evals_result()

XGBoostError: No evaluation result, `eval_set` is not used during training.

In [72]:
model_filename = f"model.joblib"
joblib.dump(model_dict, model_filename)


['model.joblib']

In [73]:
model_dict[0]

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [74]:
ml = joblib.load("model.joblib")

In [76]:
ml.get(1)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
num = df.rdd.getNumPartitions()
print("number of partion before:",num)
# df_repartitoned = df.repartition("sensor_id")
# num2 = df_repartitoned.rdd.getNumPartitions()

In [ ]:
# store_part = spark.sql(sql).repartition(spark.sparkContext.defaultParrallelism,['sensor_id']).cache()
store_part = spark.sql(sql).repartition(spark.sparkContext.defaultParallelism, ['sensor_id']).cache()


In [ ]:
store_part.explain()

In [ ]:
from pyspark.sql.types import FloatType,IntegerType,StringType,TimestampType,StructType,StructField

result_schema = StructType([
    StructField("temperature_2m", FloatType(), True),
    StructField("relative_humidity_2m", IntegerType(), True),
    StructField("apparent_temperature", FloatType(), True),
    StructField("precipitation", FloatType(), True),
    StructField("wind_speed_10m", FloatType(), True),
    StructField("wind_speed_100m", FloatType(), True),
    StructField("creation_time", TimestampType(), True),
    StructField("sensor_id", StringType(), True),
    StructField("count", IntegerType(), True),
    StructField("is_holiday", IntegerType(), True),
    StructField("consumed_unit", FloatType(), True),
    StructField("lag1", FloatType(), True),
    StructField("lag2", FloatType(), True),
    StructField("lag3", FloatType(), True),
    StructField("lag4", FloatType(), True),
    StructField("lag5", FloatType(), True),
    StructField("lag6", FloatType(), True),
    StructField("lag7", FloatType(), True),
    StructField("day", IntegerType(), True),
    StructField("hour", IntegerType(), True),
    StructField("month", IntegerType(), True),
    StructField("dayofweek", IntegerType(), True),
    StructField("quarter", IntegerType(), True),
    StructField("dayofyear", IntegerType(), True),
    StructField("weekofyear", IntegerType(), True),
    StructField("year", IntegerType(), True),
])

In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
# @pandas_udf(result_schema,PandasUDFType.GROUPED_MAP)
@pandas_udf(result_schema, PandasUDFType.GROUPED_MAP)
# def load_forecast(senor_data):
def train_test_split(df):
    try:
        print("hello")
        # total_rows = df.count()

        # test_rows = int(0.2 * total_rows)
        # train_rows = max(0, total_rows - test_rows)
        # test = df.orderBy(col("creation_time").desc()).limit(test_rows)
        # train = df.orderBy(col("creation_time").asc()).limit(train_rows)
        # return [(train,test)]
        return df
    except Exception as e:
        print("error in spliting data:",e)
# result = (store_part.groupBy("sensor_id").apply(train_test_split))

In [ ]:
# from pyspark.sql.functions import pandas_udf, PandasUDFType
# import pandas as pd
# @pandas_udf(result_schema,PandasUDFType.GROUPED_MAP)
# def train_test_split(df):
#     try:
#         print("hello")
#         return pd.DataFrame({"result": [df]})  # Example return, adjust as needed
#     except Exception as e:
#         print("error in splitting data:", e)

result = (store_part.groupBy("sensor_id").apply(train_test_split))


In [ ]:
result.cache()

In [ ]:
result.show()

In [ ]:
result.explain()

In [ ]:
result.printSchema()

In [ ]:
result.collect()

In [ ]:
spark

In [ ]:
import random

names = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Henry", "Ivy", "Jack"]
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Department", StringType(), True)
])
# List of departments
departments = ["HR", "IT", "Finance", "Marketing", "Operations"]

# Randomly choose departments for each person
data = [(name, random.choice(departments)) for name in names]

# Extend the data to have some repetitive departments
data_extended = data + data[:3]  # Repeat the first three departments

# Create a DataFrame
df = spark.createDataFrame(data_extended, schema)
df.show()

In [ ]:
df_repartitoned = df.repartition("department")
print(df_repartitoned.rdd.getNumPartitions())

In [ ]:
# def f(people):
#     for person in people:
#         print(person.Name)
# df_repartitoned.foreachPartition(f)

def f(iterator):
    for person in iterator:
        print(person.Name)

# Apply the function to each partition
df_repartitoned.foreachPartition(f)


In [ ]:
def f(iterator):
    names = [person.Name for person in iterator]
    print(names)

# Apply the function to each partition and collect the results to the driver
df_repartitoned.foreachPartition(f)


In [ ]:
names = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Henry", "Ivy", "Jack"]
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Department", StringType(), True)
])
departments = ["HR", "IT", "Finance", "Marketing", "Operations"]
data = [(name, random.choice(departments)) for name in names]
df = spark.createDataFrame(data_extended, schema)
# df.show()

# Repartition the DataFrame based on the "Department" column
df_repartitioned = df.repartition("Department")
print(df_repartitioned.rdd.getNumPartitions())
# Print names within each partition using collect() and a loop
# if df_repartitioned.rdd.isEmpty():
#     print("DataFrame is empty")
# else:
#     # Get names from each partition using collect()
#     partitions = df_repartitioned.rdd.collect()

#     # Loop through partitions and print names
#     for partition in partitions:
#         partition.show()
#         names = [person.Name for person in partition]
#         print(f"Names in partition: {names}")


In [ ]:
partitions

In [ ]:
df_repartitioned.show()

In [ ]:
df.show()

In [ ]:
total_consumption_df = df.groupBy("sensor_id").agg(sum("consumed_unit").alias("total_consumption"))
total_consumption_df.show()

In [ ]:
df_repartitioned = df.repartition("sensor_id")
# Define a UDF to encapsulate your calculations
def analyze_timeseries(data):
  # Your time series analysis logic goes here
  # Example: Calculate mean consumption
  mean_consumption = data.select("consumed_unit").mean()  # Access the DataFrame within the UDF
  return mean_consumption

analyze_timeseries_udf = spark.udf.register("analyze_timeseries", analyze_timeseries)

# Apply the UDF to each partition (sensor)
analyzed_df = df_repartitioned.withColumn("analysis_result", analyze_timeseries_udf(col("consumed_unit")))
analyzed_df.show()


In [ ]:
from pyspark.sql import SparkSession, functions as F

partitioned_df = df.repartition("sensor_id") 

# Define a UDF to encapsulate calculations (using PySpark functions for clarity)
def analyze_timeseries(data):
  print(data)
  mean_consumption = F.mean(data["consumed_unit"])
  return mean_consumption

analyze_timeseries_udf = spark.udf.register("analyze_timeseries", analyze_timeseries)

# Apply the UDF to each partition (sensor), ensuring type safety
analyzed_df = partitioned_df.withColumn("analysis_result", analyze_timeseries_udf(col("consumed_unit").cast("double")))

# View the results
analyzed_df.show()


In [ ]:
# grouped_data = df.groupBy("sensor_id")
df_repartitoned = df.repartition

def train_test_split(df):
    try:

        total_rows = df.count()
        test_rows = int(0.2 * total_rows)
        train_rows = max(0, total_rows - test_rows)
        print(total_rows,test_rows,train_rows)
        test = df.orderBy(col("creation_time").desc()).limit(test_rows)
        train = df.orderBy(col("creation_time").asc()).limit(train_rows)
        return train,test
    except Exception as e:
        print("error in spliting data:",e)
train_test_splits = grouped_data.flatMap(lambda group: train_test_split(group[1]))

train_test_splits = grouped_data.apply(train_test_split)

# Collect train-test splits
train_test_splits = train_test_splits.collect()

In [ ]:
schema = StructType([
    StructField("value", StringType(), True),
    StructField("creation_time", StringType(), True),
    StructField("sensor_id", StringType(), True)])

def train_test_split(key, iterator):
    df = pd.DataFrame(list(iterator), columns=df.columns)
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    yield train_df, test_df

# Apply train-test split function to each group in parallel
train_test_splits = df.groupBy("sensor_id").applyInPandas(train_test_split, schema)

# Collect train-test splits
train_test_splits_list = train_test_splits.tail(5)

In [ ]:
# Show the first few rows of each train-test split DataFrame
for train_test_split_df in train_test_splits.take(5):
    train_test_split_df.show()


In [ ]:
schema_file_path = "schema.yaml"
# Validate the data against the schema
validated_df = reading_and_validate_data(spark, data, schema_file_path)

# Check if validation was successful
if validated_df is not None:
    # validated_df.show()
    df = validated_df.select("creation_time","sensor_id","consumed_unit")
    # df = validated_df
    df.show()
    df.createOrReplaceTempView("data")
    spark.sql("""
    select sensor_id,sum(consumed_unit) as units, count(sensor_id) as data_count
    from data
    group by sensor_id
    order by data_count desc
    """).show()
else:
    print("Data validation failed.")


In [ ]:
pandas_df = sdf.toPandas()

In [ ]:
pandas_df

In [ ]:
pandas_df.info()

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

test_size = 24 * 6 * 4  # 7 days * 24 hours/day
gap = 24*4  # 1 day * 24 hours/day

total_size = test_size + gap

tss = TimeSeriesSplit(n_splits=3, test_size=total_size, gap=gap)

for train_idx, val_idx in tss.split(pandas_df):
    train_data = pandas_df.iloc[train_idx]
    test_data = pandas_df.iloc[val_idx]

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
plt.rcParams['figure.figsize'] = 18,6
plt.plot(test_data["creation_time"],test_data['consumed_unit'])

In [ ]:
sdf

In [ ]:
sdf.show()

In [ ]:
train,test = sdf.randomSplit([0.8,0.2])

In [ ]:
train.select(count("sensor_id")).show()


In [ ]:
test.select(count("sensor_id")).show()


In [ ]:
test.show()

In [ ]:
# test_rows = 7 * 24 * 4

def train_test_split(df):
    try:
        total_rows = df.count()
        test_rows = int(0.2 * total_rows)
        train_rows = max(0, total_rows - test_rows)
        test = df.orderBy(col("creation_time").desc()).limit(test_rows)
        train = df.orderBy(col("creation_time").asc()).limit(train_rows)
        return train,test
    except Exception as e:
        print("error in spliting data:",e)
        
def train_test_split_multi_timeseries(df):
    window_spec = window.orderBy(col("creation_time").asc())

    # Calculate total and test rows (adjust test_size as needed)
    total_rows = df.count()
    test_rows = int(0.2 * total_rows)

    # Ensure train_rows is not negative
    train_rows = max(0, total_rows - test_rows)

    # Apply window and filter for training and testing data
    df_with_window = df.withColumn("window_id", window_spec)    
    train = df_with_window.filter(col("window_id") <= train_rows)
    test = df_with_window.filter(col("window_id") > train_rows).orderBy(col("creation_time").desc())
    return train, test
    window_spec = window.orderBy(col("creation_time").asc())  # Order by ascending time
    window_spec = window.orderBy(col("creation_time").asc())  # Order by ascending creation time

    # Calculate total and test rows (adjust test_size as needed)
    total_rows = df.count()
    test_rows = int(0.2 * total_rows)

    # Ensure train_rows is not negative
    train_rows = max(0, total_rows - test_rows)  # Handle potential negative train_rows

    # Apply window and filter for training and testing data
    df_with_window = df.withColumn("window_id", window_spec)
    train = df_with_window.filter(col("window_id") <= train_rows)
    test = df_with_window.filter(col("window_id") > train_rows).orderBy(col("creation_time").desc())  # Descending order for test

    return train, test

In [ ]:
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

# Define the split ratios (e.g., 80% training, 20% testing)
train_ratio = 0.8
test_ratio = 1 - train_ratio

# Define a window partitioned by sensor_id and ordered by creation_time
window_spec = Window.partitionBy("sensor_id").orderBy("creation_time")

# Add row numbers within each partition (sensor_id)
sdf_with_row_number = sdf.withColumn("row_number", row_number().over(window_spec))

# Calculate the split row numbers for each sensor group
split_row_number = (col("max_row_number") * train_ratio).cast("int")

# Split the data into training and testing sets based on row numbers
train_data = sdf_with_row_number.filter(col("row_number") <= split_row_number)
test_data = sdf_with_row_number.filter(col("row_number") > split_row_number)

# Drop the row_number column from the final DataFrames
train_data = train_data.drop("row_number")
test_data = test_data.drop("row_number")

# Show the number of rows in each set
print("Number of rows in training set:", train_data.count())
print("Number of rows in testing set:", test_data.count())


In [ ]:
train, test = train_test_split_multi_timeseries(sdf)

In [ ]:
train.select(count("sensor_id")).show()
train.show()

In [ ]:
test.select(count("sensor_id")).show()
test.show()

In [ ]:
from pyspark.sql import SparkSession, Window, functions as F


def train_test_split(df):
    """Splits a DataFrame into train and test sets based on time series."""

    try:
        window_spec = Window.orderBy(F.col("creation_time").asc()) \
                           .partitionBy("sensor_id") \
                           .rowsBetween(Window.unboundedPreceding, Window.currentRow)

        df = df.withColumn("row_number", F.row_number().over(window_spec))

        total_rows = df.count()
        test_rows = int(0.2 * sdf.count())
        train_rows = total_rows - test_rows

        # Ensure train_rows is not negative
        if train_rows < 0:
            raise ValueError("Test rows exceed total rows. Adjust the test size.")

        test = df.orderBy(col("creation_time").desc()).limit(test_rows)
        train = df.orderBy(col("creation_time").asc()).limit(train_rows)
        total_rows = df.count()
        train_threshold = total_rows - test_rows

        train_df = df.where(F.col("row_number") <= train_threshold)
        test_df = df.filter(F.col("row_number") > train_threshold)

        return train_df, test_df
    except Exception as e:
        print("Error in splitting data:", e)

# Split the data into train and test sets
train_df, test_df = train_test_split(df.copy())  # Make a copy to avoid modifying original DataFrame

# Now you can use train_df and test_df for further analysis or machine learning tasks

# Stop the SparkSession
spark.stop()


In [ ]:
df.select(count_distinct("sensor_id")).show()

In [ ]:
sdf = df

In [ ]:
window_spec = Window.partitionBy("sensor_id")
# data = sdf.withColumn("total_units"),

In [ ]:
result_df = sdf.withColumn("total_units", sum("consumed_unit").over(window_spec)) \
               .withColumn("average_units", avg("consumed_unit").over(window_spec))

# Display the result
result_df.show()

In [ ]:
sdf
test_data_size = 24 * 4 * 7  # 24 hours * 4 quarters per hour * 7 days

# Sort the DataFrame by creation_time
sorted_df = df.sort(col("creation_time"))

# Split the data into training and testing sets
train_data = sorted_df.limit(sorted_df.count() - test_data_size)
test_data = sorted_df.limit(test_data_size)

# Show the schemas of the train and test DataFrames
train_data.printSchema()
test_data.printSchema()

In [ ]:
from pyspark.sql import Window

def train_test_split_per_sensor(df, test_size_days=7, gap_days=1):
    """
    Perform train-test split for each sensor_id using window functions.

    Parameters:
    - df: PySpark DataFrame containing the time series data with columns "creation_time" and "sensor_id".
    - test_size_days: Size of the test set in days.
    - gap_days: Gap between test sets in days.

    Returns:
    - train_data: PySpark DataFrame containing the training data.
    - test_data: PySpark DataFrame containing the test data.
    """
    # Define window specification partitioned by sensor_id and ordered by creation_time
    window_spec = Window.partitionBy("sensor_id").orderBy("creation_time")

    # Calculate test and train timestamps using lag function
    df_with_timestamps = df.withColumn("test_end_timestamp", F.lag("creation_time", test_size_days).over(window_spec)) \
                           .withColumn("test_start_timestamp", F.expr(f"test_end_timestamp - INTERVAL {test_size_days} DAYS")) \
                           .withColumn("train_end_timestamp", F.expr(f"test_start_timestamp - INTERVAL {gap_days} DAYS")) \
                           .withColumn("train_start_timestamp", F.first("creation_time").over(window_spec))

    # Perform train-test split based on calculated timestamps
    train_data = df_with_timestamps.filter((F.col("creation_time") >= F.col("train_start_timestamp")) & (F.col("creation_time") < F.col("train_end_timestamp")))
    test_data = df_with_timestamps.filter((F.col("creation_time") >= F.col("test_start_timestamp")) & (F.col("creation_time") < F.col("test_end_timestamp")))

    return train_data, test_data

# Example usage:
train_data, test_data = train_test_split_per_sensor(df)


In [ ]:
train_data.show()

In [ ]:
from pyspark.sql import functions as F

test_size_intervals = 7 * 24 * 4  # 7 days * 24 hours/day * 4 intervals/hour (15 minutes)
gap_intervals = 24 * 4  # 1 day * 24 hours/day * 4 intervals/hour (15 minutes)

# Convert intervals to seconds
test_size_seconds = test_size_intervals * 15 * 60
gap_seconds = gap_intervals * 15 * 60

# Function to perform train-test split for each sensor_id
def split_train_test_per_sensor(rows):
    for sensor_id, data_per_sensor in rows:
        min_timestamp = data_per_sensor.select(F.min("creation_time")).first()[0]
        max_timestamp = data_per_sensor.select(F.max("creation_time")).first()[0]
        
        # Calculate test start and end timestamps
        test_end_timestamp = max_timestamp
        test_start_timestamp = test_end_timestamp - test_size_seconds
        
        # Calculate train start and end timestamps
        train_end_timestamp = test_start_timestamp - gap_seconds
        train_start_timestamp = min_timestamp
        
        # Perform train-test split for the current sensor_id
        train_data = data_per_sensor.filter((F.col("creation_time") >= train_start_timestamp) & (F.col("creation_time") < train_end_timestamp))
        test_data = data_per_sensor.filter((F.col("creation_time") >= test_start_timestamp) & (F.col("creation_time") < test_end_timestamp))
        
        yield sensor_id, train_data, test_data

# Perform groupBy operation on sensor_id and apply the split_train_test_per_sensor function
train_test_per_sensor = df.groupBy("sensor_id").mapPartitions(split_train_test_per_sensor)

# Collect the results if needed
results = train_test_per_sensor.collect()


In [ ]:
from pyspark.sql.functions import window, col, lit

def timeseries_train_test_split_sensor(df, training_ratio, test_size, gap_size):
  """
  This function splits data for each unique sensor_id into training and testing sets 
  based on timestamps.

  Args:
      df: PySpark DataFrame containing time series data
      training_ratio: Proportion of data allocated to training (0.0 to 1.0)
      test_size: Number of time steps for the testing set
      gap_size: Number of time steps between the training and testing sets

  Returns:
      A PySpark DataFrame containing sensor_id, training data, and testing data
  """

  # Calculate training data size based on ratio
  training_size = int(df.count() * training_ratio)

  # Window function to calculate the training end timestamp
  window_spec = window.orderBy("timestamp").partitionBy("sensor_id").rowsBetween(-test_size - gap_size, -1)
  df_with_training_end_ts = df.withColumn("training_end_ts", window_spec.end())

  # Filter training and testing data based on timestamp
  training_data = df.where(col("timestamp") <= col("training_end_ts"))
  testing_data = df.where(col("timestamp") > col("training_end_ts"))

  # Ensure training data covers training_size rows (optional)
  training_data = training_data.limit(training_size)

  # Combine sensor_id, training, and testing data into a single DataFrame
  split_data = training_data.select("sensor_id", "timestamp", col("*").alias("training"))\
               .union(testing_data.select("sensor_id", "timestamp", col("*").alias("testing")))

  return split_data

# Split data for each sensor
split_data = df.groupBy("sensor_id").applyInPartitions(timeseries_train_test_split_sensor,lit(0.8), lit(24 * 7), lit(24))


In [ ]:
from pyspark.sql.functions import window, col, lit

def timeseries_train_test_split_sensor(iterator, training_ratio, test_size, gap_size):
    """
    This function splits data for each unique sensor_id into training and testing sets 
    based on timestamps (for mapPartitions).

    Args:
        iterator: Iterator over a partition of the grouped DataFrame
        training_ratio: Proportion of data allocated to training (0.0 to 1.0)
        test_size: Number of time steps for the testing set
        gap_size: Number of time steps between the training and testing sets

    Yields:
        Rows containing sensor_id, training data, and testing data
    """
    # Calculate training data size based on ratio
    training_size = int(df.count() * training_ratio)

    # Window function to calculate the training end timestamp
    window_spec = window.orderBy("timestamp").partitionBy("sensor_id").rowsBetween(-test_size - gap_size, -1)
    df_with_training_end_ts = df.withColumn("training_end_ts", window_spec.end())

    # Filter training and testing data based on timestamp
    training_data = df.where(col("timestamp") <= col("training_end_ts"))
    testing_data = df.where(col("timestamp") > col("training_end_ts"))

    # Ensure training data covers training_size rows (optional)
    training_data = training_data.limit(training_size)

    # Combine sensor_id, training, and testing data into a single DataFrame
    split_data = training_data.select("sensor_id", "timestamp", col("*").alias("training"))\
                 .union(testing_data.select("sensor_id", "timestamp", col("*").alias("testing")))

    # Yield rows from the combined DataFrame
    for row in split_data.rdd.collect():
      yield row

# Split data for each sensor
split_data = df.groupBy("sensor_id").mapPartitions(timeseries_train_test_split_sensor,lit(0.8), lit(24 * 7), lit(24))


In [ ]:
from pyspark.sql.functions import udf, col, lit, struct, window

def train_test_split_udf(data, training_ratio, test_size, gap_size):
  """
  This UDF splits data for a single sensor into training and testing sets 
  based on timestamps.

  Args:
      data: Sub-DataFrame for a single sensor_id
      training_ratio: Proportion of data allocated to training (0.0 to 1.0)
      test_size: Number of time steps for the testing set
      gap_size: Number of time steps between the training and testing sets

  Returns:
      A tuple containing two DataFrames (training_data, testing_data)
  """

  # Calculate training data size based on ratio
  training_size = int(data.count() * training_ratio)

  # Order data by timestamp
  data = data.orderBy("timestamp")  # Assuming you have a "timestamp" column

  # Calculate end timestamp for training data (considering gap)
  training_end_ts = data.select(window.last("timestamp")).rdd.map(lambda r: r[0]).first() - \
                    lit(test_size + gap_size)

  # Filter training and testing data based on timestamp
  training_data = data.where(col("timestamp") <= training_end_ts)
  testing_data = data.where(col("timestamp") > training_end_ts)

  # Ensure training data covers training_size rows (optional)
  training_data = training_data.limit(training_size)

  return training_data, testing_data

# Define UDF
split_udf = udf(train_test_split_udf, structType=[training_data.schema, testing_data.schema])

# Group by sensor_id and apply UDF
split_data = df.groupBy("sensor_id").apply(split_udf, lit(0.8), lit(24 * 7), lit(24))

# Extract training and testing DataFrames from the split data
split_data = split_data.select("sensor_id", split_udf.alias("data").explode())
training_data = split_data.select("sensor_id", "data.training")
testing_data = split_data.select("sensor_id", "data.testing")


In [ ]:

def train_data(spark):
    """
    processing data
    """
    try:
        path1 = "D:\Projects\Spark\data.csv"       
        spark_df = spark.read.csv(path1,header= True)
        spark_df.show()
        spark_df.createOrReplaceTempView("data")
        spark.sql("""
        select count(*)
        from data 

        """).show()
    except Exception as e:
        print("error in traininig:",e)

In [ ]:
spark = initiate_spark()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)  # Enable eager evaluation
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 20)   # Set max number of rows to display
spark.conf.set("spark.sql.repl.eagerEval.maxNumColumns", 200)  # Set max number of columns to display
spark.conf.set("spark.sql.repl.eagerEval.truncate", 0)  # Disable truncation

if spark:
    try:
        # training data
        train_data(spark)

        # display(df.select('sensor_id'))
    finally:
        # Terminate SparkSession
        # terminate_spark(spark)
        pass